<a href="https://colab.research.google.com/github/jiunny/NH_DACON/blob/feature%2Fjiunny/mecab_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Mecab 사용자 사전 추가



In [ ]:
# mecab 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

# 종성 여부 판단 - nnp사전에 추가할 때 종성여부를 넣어주어야 함
!pip install jamo

## 데이터 불러오기

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/data/news_test.csv') 

## 시간 측정 시작

In [ ]:
import time
start = time.time()

## 1. Load Module / Data
- colab과 google drive 연동




In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

## Library 불러오기

In [ ]:
import numpy as np
import re
from konlpy.tag import Mecab
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

## Tokenizer,Model,외부 데이터 불러오기

In [ ]:
def split_tokenizer(text):
    tokens_ko = text.split()
    return tokens_ko

# tf-idf 로딩
with open( "/content/drive/MyDrive/tfidf", "rb" ) as file:
    tfidf_vect = pickle.load(file)

# lgbm model 로딩
with open( "/content/drive/MyDrive/lgbm", "rb" ) as file:
    tune_lgbm = pickle.load(file)

# 외부데이터 로딩
with open( "/content/drive/MyDrive/cover", "rb" ) as file:
    cover = pickle.load(file)

## 형태소 분석 + 전처리

### mecab 사용자 사전 추가


In [ ]:
# 초기에 만들어져 있던 nnp사전
with open("/tmp/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()

# dict_ls는 넣고싶은 nnp 목록(list 형태)
dict_ls = '코로나19, 확진자, 관련주, 순매수, 상한가, 이데일리, 저가, 줄상한가,급등, \
증권사, 1위, 수혜주,文대통령, 상승세, 하락세, 렘데시비르, \
삼성전자, 골든클럽, 신라젠, 실시간, 하이스탁론, 치료제,알테오젠, \
임상3상, 임상, 5G, CGV, 대장주, 장중, 본격화, 유상증자, 에이치엘비, \
렘데시비르, 카톡방,세일, 순 매도, 하이스탁론 ,코로나 , 테크노밸리, 코미팜,\
익스프리언스,비과세,삼성전자,lg전자,확진자,헬스케어,황제주,실적,재배포,전국적,2020년,\
홈쇼핑,비대면,홈쇼핑사,씽크풀스탁론,마이크론, 추천주, 대박주, 스탁론,셀트리온그룹,아시아나,한화솔루션,솔루션,핫이슈,이슈,\
베팅,NH투자,통합당,언택트,에스오일,바이오시밀러,셀트리온,협력사,LG디스플레이,SK이노베이션,SK디앤디,케이피에스,저가항공'.replace(" ", '').split(',')

# 종성여부를 판단하는 사용자 정의함수
from jamo import h2j, j2hcj 
def get_jongsung_TF(sample_text): 
    sample_text_list = list(sample_text) 
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word))) 
    last_jamo = last_word_jamo_list[-1] 
     
    jongsung_TF = "T" 
    
    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']: 
        jongsung_TF = "F"

    return jongsung_TF

# nnp추가 형식에 맞추어서 dict_ls 추가과정
for word in dict_ls: 
    jongsung_TF = get_jongsung_TF(word) 
    line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word) 
    file_data.append(line)

with open("/tmp/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv", 'w', encoding='utf-8') as f: 
    for line in file_data: 
        f.write(line)

# make 파일을 통해서 최종 compile
!bash /tmp/mecab-ko-dic-2.1.1-20180720/tools/add-userdic.sh
%cd /tmp/mecab-ko-dic-2.1.1-20180720/
!./configure
!make
!make install

In [ ]:
mecab = Mecab()
test.columns = ['n_id','date','title','content','ord','id_']

## title,content 변수 전처리

In [ ]:
def fn_clean_var(df):

    try:
        def cleansing(text):
            text = re.sub("(=*([가-힣]{2,4}|[가-힣]{6}) 기자(... [가-힣]{2,3}기자|[^가-힣]|$)=*|로봇 기자)",r' reporter\3 ',text) # 000기자 -> reporter로 변환
            text = re.sub('(‘|\")(.+)(’|\")',r" '\2' ", text)                     # ' " ’ -> 로  변환
            text = re.sub('【(.+)】',r' [\1] ', text)                              #【( 등과 같은 괄호 '[ ]' 로 변환
            text = re.sub('[0-9]+.[0-9]+%|[0-9]+.[0-9]+%|[0-9]+%',' perc ', text)  #12% , 00% -> 'perc'로  변환
            text = re.sub('\w+\([0-9]+\)',' stock ', text)    # 삼성전자[12345] , 카카오[234421] -> 'stock' 로 변환
            text = re.sub('(http|ftp|https)://((\w+\.|\w+/|\w+|(\?|=|\&)\w+)+)', ' link ', text)  #url -> 'link' 로 변환
            text = re.sub('[0-9]+-[0-9]+|02-[0-9]+-[0-9]+',' phone ', text)        #000-0000-0000 -> 'phone'로 변환
            text = re.sub('TOP [0-9]+|TOP[0-9]+',' rank', text)                    # TOP 1, TOP 10 -> 'rank'로 변환
            text = re.sub('([0-9]+|[0-9]+\s|\w*백|\w*천)(억|만원|조)',' money ', text)  # 3백만원, 3천억, 3천조 -> 'money' 로변환
            text = re.sub('[0-9]+:[0-9]+|[0-9]+시 [0-9]분',' time ', text)         # 12:12 , 12시 12 분 -> 'time'로 변환
            text = re.sub('[0-9]+월|[0-9]+개월|[0-9]+ 개월',' month ', text)       # 00월 , 00개월, 00 개월 - >'month' 로 변환
            text = re.sub('[0-9]+일',' day ', text)                                # [0-9]일 -> 'day'로 변환
            text = re.sub('[0-9]+년|2020',' year ', text)                          # 00년 ,2020 -> 'year'로 변환
            text = re.sub('([0-9]+|[0-9]+\s)배',' bae ', text)                     # 00배 , 00 배 -> 'bae'로 변환
            text = re.sub('코로나19|코로나 19',' 코로나 ', text)                   # 코로나19, 코로나 19 -> '코로나' 로 변환
            text = re.sub('(\[\w+)(=)(\w+\])',r'\1 \2 \3',text )                # 해럴드경제(대전)=홍길동기자 ->해럴드경제 (대전) 홍길동기자        
            text = re.sub("(\w+)(\")(\w+)",r"\1 \2 \3",text)                    
            text = re.sub('(&nbsp;)|(&nbsp)|(&#[0-9]+;)|(fffff;)|\x03',"",text) # css 공통 문법 제거

            if re.search('[a-zA-Z]+-[a-zA-Z]+(:|;)',text)!=None:
                text = " ".join(re.findall('[가-힣]+|[0-9]+[가-힣]+',text)) #css 문법(font-family 등)이 있는 경우에는 한글/숫자+한글인 것만 추출

            text = re.sub('[-_+]|돋움|&[a-zA-Z]{2,100};'," ",text)          # 돋움이라는 font가 들어있는 경우는 css에서 제거하기 힘들어서 별도로 제거 + &quot같은 것들도 제거
            text = re.sub('(\w+)(\s\.\.+|\.\.+)',r"\1 Ellipsis ",text)     # ... , ..  -> 'Ellipsis' 로 변환
            text = re.sub('\.',"",text)                                    # 마침표 제거 
            text = re.sub('\s\s+'," ",text)                                # 공백 2개이상 -> 한개로

            if re.search('\S',text)==None:                                          # 텍스트가 비어있으면 'blank'로 채움
                text = 'blank' 

            return text

        # title 특수기호 추가
        def feature(text):
            candidates = ['!','◆','■','▶','★','↑','↓','?']
            for candidate in candidates:
                if candidate in text:
                    # 특수 기호가 있으면 1
                    return 1
            # 없으면 0
            return 0  

        class mecab_:

            def __init__(self,text):
                self.result = mecab.pos(text)
            
            def fn_append_mecab(self):     # 형태소중 '조사'를 제외하고 clean_word에 추가.
                clean_word = []
                for word,tag in self.result:
                    if not tag.startswith('J') :
                        clean_word.append(word)
                
                return " ".join(clean_word)        
        
            def fn_last_pos(self):          #첫번째 단어 형태소 추출
                try: 
                    last_pos = self.result[-1][1]
                    return last_pos
                except Exception as e:
                    last_pos = mecab.pos(re.escape(text))[-1][1]
                    return last_pos

            def fn_first_pos(self):           # 마지막 단어 형태소 추출
                try: 
                    first_pos = self.result[0][1]
                    return first_pos
                except Exception as e:
                    first_pos = mecab.pos(re.escape(text))[0][1]
                    return first_pos

        df['clean_content'] = df['content'].apply(lambda x: cleansing(x))   # 정규표현식을 이용한 텍스트 정제
        df['clean_title'] = df['title'].apply(lambda x: cleansing(x))       # 정규표현식을 이용한 텍스트 정제

        df['class_cont'] = df['clean_content'].apply(lambda x: mecab_(x)) #mecab_pos 돌린것 저장해두기
        df['class_tit'] = df['clean_title'].apply(lambda x: mecab_(x))  #mecab_pos 돌린것 저장해두기(title))
        
        df['mecab_'] = df['class_cont'].apply(lambda x: x.fn_append_mecab())          #mecab_ 변수 생성

        df['last_pos_content'] = df['class_cont'].apply(lambda x: x.fn_last_pos())   # content의 마지막 단어 형태소 추가
        df['first_pos_content'] = df['class_cont'].apply(lambda x: x.fn_first_pos()) # content의 첫번째 단어 형태소 추가

        df['last_pos_title'] = df['class_tit'].apply(lambda x: x.fn_last_pos())    # title의 마지막 단어 형태소 추가
        df['first_pos_title'] = df['class_tit'].apply(lambda x: x.fn_first_pos())  # title의 첫번째 단어 형태소 추가.

        df['title_feature'] = df['clean_title'].apply(lambda x :feature(x))     #title_feature 변수 추가

        del df['class_cont']
        del df['class_tit']

        return df

    except Exception as e:
        print('ERRER OCCUR '+e)



test_df = fn_clean_var(test)

In [ ]:
# new 변수추가 
test_df['new'] =  'FC'+test_df['first_pos_content']+" "+'FT'+test_df['first_pos_title']+" "+test_df['mecab_'] + " LC"+test_df['last_pos_content']+" LT"+test_df['last_pos_title']

#ord 변수추가
test_df['ord_div'] = test_df['ord'].div(test_df.groupby('n_id')['ord'].transform(lambda x: x.max()))

#counts 변수추가
counts = pd.DataFrame({'content':test_df.groupby('content').size().index,
                       'counts' :test_df.groupby('content').size().values})
test_df = pd.merge(test_df,counts,how='left')


tfidf_matrix_test = tfidf_vect.transform(test_df['new'])
sparse_matrix = sp.csr_matrix(test_df[['ord_div','counts','title_feature']].T)
tfidf_matrix_test = sp.hstack((sparse_matrix.T,tfidf_matrix_test))

## 예측

In [ ]:
pre = tune_lgbm.predict(tfidf_matrix_test)
test_df['pred'] = pre
test_df.loc[test_df.content.apply(lambda x: x in cover),'pred'] = 1

In [ ]:
# 제출 파일 생성
sample_submission  = pd.DataFrame(test_df[['id_','pred']])
sample_submission.columns = [['id','pred']]

In [ ]:
print(time.time() - start)

## 제출

In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/final_check.csv',index = False)